In [7]:
alpha_name = "AlphaNet_8Input_Float2000_Shift_1_Sequence_20_10dRet"
start_date = '2019-01-01'
end_date = '2021-06-01'
change = 10

In [8]:
import time
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
import numpy as np
import pandas as pd
import DataAPI
import os
input_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun/' + alpha_name
output_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun/'

alpha_list = []
trading_list = DataAPI.get_trading_days(start_date, end_date)
count = 0
for day in trading_list:
    file = os.path.join(input_path, str(day.year), day.strftime('%Y%m%d') + '.csv')
    try:
        df = pd.read_csv(file, dtype={0: str})
    except FileNotFoundError:
        count += 1
        continue
    flag = count % change
    path_output = os.path.join(output_path, str(alpha_name + "_" + str(flag)), str(day.year), day.strftime('%Y%m%d') + '.csv')
    alpha_list.append(str(alpha_name + "_" + str(flag)))
    if not os.path.exists(path_output):
        os.makedirs(os.path.dirname(path_output), exist_ok=True)
    df.to_csv(path_output, index=False, header=False)
    count += 1

In [9]:
pd.set_option('expand_frame_repr', False)
config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
print('Loading the configuration from ' + config_path)
configs = namespace.load_namespace(config_path)
FT = FeatureAnalysis(configs, feature_path=r"/home/wuwenjun/factor_lib")
alpha_list = list(set(alpha_list))
print(alpha_list)
FT.load_feature_from_file(alpha_list, start_date, end_date, universe='Investable',
                                       timedelta=None, transformer=norm.standard_scale)
FT.load_return_data()

writing_path = r'/home/wuwenjun/Desktop/%s_%ddChange.txt' % (alpha_name,change)
f = open (writing_path,'w')
print("\n",file=f)
f.close()
for alpha_name in alpha_list:
    FT.get_intersection_ic(feature_name=alpha_name, truncate_fold=None, method='spearman',
                                     period=('1d', '3d', '5d'))
    ic_flag, trading_direction = FT.test_ic(alpha_name, verbose=False)
    df, df_all = FT.get_ic_summary_by_month(num=6)
    
    if trading_direction == -1:
        negative = True
    else:
        negative = False
    f = open (writing_path,'a')
    print("*"*100,end="\n"*3,file=f)
    print(alpha_name,end = "\n"*3,file=f)
    print(df,end="\n"*3,file = f)
    a = FT.get_top_return(alpha_name, negative= False, trade_type='long-only', transmission_rate=0.00025,
                            tax_rate=0.001, verbose=True,bt_price = "vwap",trade_period=6)
    print(a[1],end = "\n"*3+"*"*100 ,file = f)
    f.close()

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


Loading Feature...:   2%|█▋                                                                                       | 109/5840 [00:00<00:07, 737.91it/s]

['AlphaNet_8Input_Float2000_Shift_1_Sequence_20_10dRet_6', 'AlphaNet_8Input_Float2000_Shift_1_Sequence_20_10dRet_5', 'AlphaNet_8Input_Float2000_Shift_1_Sequence_20_10dRet_8', 'AlphaNet_8Input_Float2000_Shift_1_Sequence_20_10dRet_1', 'AlphaNet_8Input_Float2000_Shift_1_Sequence_20_10dRet_4', 'AlphaNet_8Input_Float2000_Shift_1_Sequence_20_10dRet_2', 'AlphaNet_8Input_Float2000_Shift_1_Sequence_20_10dRet_9', 'AlphaNet_8Input_Float2000_Shift_1_Sequence_20_10dRet_3', 'AlphaNet_8Input_Float2000_Shift_1_Sequence_20_10dRet_7', 'AlphaNet_8Input_Float2000_Shift_1_Sequence_20_10dRet_0']


Calculating IC value...: 100%|█████████████████████████████████████████████████████████████████████████████████████| 487/487 [00:00<00:00, 686.90it/s]



IC table for feature "AlphaNet_8Input_Float2000_Shift_1_Sequence_20_10dRet_6" (mean std IR):
+-------------------+------------------+------------------+------------------+------------------+------------------+
|      period       |  IC_AlphaNet_1d  |  IC_AlphaNet_2d  |  IC_AlphaNet_3d  |  IC_AlphaNet_4d  |  IC_AlphaNet_5d  |
+===================+==================+==================+==================+==================+==================+
| 20190110-20190630 |  5.97 5.81 1.03  |  8.07 8.09 1.00  |  6.98 9.77 0.71  |  6.10 9.07 0.67  |  6.08 9.17 0.66  |
+-------------------+------------------+------------------+------------------+------------------+------------------+
| 20190701-20191231 |  2.85 4.82 0.59  |  5.72 6.51 0.88  |  4.85 6.70 0.72  |  4.70 6.33 0.74  |  5.40 6.87 0.79  |
+-------------------+------------------+------------------+------------------+------------------+------------------+
| 20200101-20200630 |  2.22 8.17 0.27  |  5.02 5.46 0.92  |  6.51 5.58 1.17  |  6.73 6.

...


AlphaNet_8Input_Float2000_Shift_1_Sequence_20_10dRet_7 back-test result for long-only mode:

Back test summary: "long-only" mode
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
|      period       |  stock_num  |  return  |  ret_std  |  ret_year  |  sharpe  |  win_ratio  |      max_dd|period      |  max_dd_day|date  |  turnover  |
+===================+=============+==========+===========+============+==========+=============+=========================+===================+============+
| 20190102-20190630 |     116     |  -0.54   |   5.69    |   -1.11    |  -0.21   |    57.26    | 3.99(20190110-20190408) |  1.13(20190129)   |    8.59    |
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
| 20190701-20191231 |     121     |   1.97   |   5.01    |    3.80    |   0.78   |    57.6

Calculating IC value...: 100%|█████████████████████████████████████████████████████████████████████████████████████| 487/487 [00:00<00:00, 655.10it/s]



IC table for feature "AlphaNet_8Input_Float2000_Shift_1_Sequence_20_10dRet_0" (mean std IR):
+-------------------+------------------+------------------+------------------+------------------+------------------+
|      period       |  IC_AlphaNet_1d  |  IC_AlphaNet_2d  |  IC_AlphaNet_3d  |  IC_AlphaNet_4d  |  IC_AlphaNet_5d  |
+===================+==================+==================+==================+==================+==================+
| 20190102-20190630 |  3.92 5.26 0.75  |  5.40 8.76 0.62  |  5.14 8.87 0.58  |  6.06 8.47 0.72  |  5.32 9.35 0.57  |
+-------------------+------------------+------------------+------------------+------------------+------------------+
| 20190701-20191231 |  4.09 6.01 0.68  |  4.01 4.12 0.97  |  4.38 6.25 0.70  |  4.64 6.23 0.74  |  3.81 5.84 0.65  |
+-------------------+------------------+------------------+------------------+------------------+------------------+
| 20200101-20200630 |  3.62 6.29 0.58  |  3.57 5.81 0.61  |  5.11 5.96 0.86  |  4.76 5.

Getting BT result for date: 20210531...: 100%|██████████████████████████████████████████████████████████████████████| 584/584 [00:06<00:00, 91.66it/s]



AlphaNet_8Input_Float2000_Shift_1_Sequence_20_10dRet_0 back-test result for long-only mode:

Back test summary: "long-only" mode
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
|      period       |  stock_num  |  return  |  ret_std  |  ret_year  |  sharpe  |  win_ratio  |      max_dd|period      |  max_dd_day|date  |  turnover  |
+===================+=============+==========+===========+============+==========+=============+=========================+===================+============+
| 20190102-20190630 |     117     |   7.47   |   6.21    |   15.86    |   2.51   |    56.41    | 4.25(20190116-20190408) |  0.97(20190308)   |    8.69    |
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
| 20190701-20191231 |     121     |   1.83   |   4.95    |    3.53    |   0.73   |    52.8